In [11]:
%reload_ext autotime
import pandas as pd
import plotly.io as pio
import plotly.express as px # Plotting
from tqdm import tqdm
tqdm.pandas()
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
pio.renderers.default = 'notebook'
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 100)

In [2]:
df = pd.read_excel("LLM_results.xlsx")
df = df[(df.timeLeft == "closed") & (df.goal > 0)]
df.sentiment = df.sentiment.astype(float)
df.truth = df.truth.astype(float)
df.urgency = df.urgency.astype(float)
df["Target_completion"] = df.amountRaised / df.goal * 100
df

,Index,title,pitch,hero,uri,amountRaised,goal,timeLeft,action,actionUri,start,end,n_questions,location,n_donors,description,use_of_funds,whos_involved,moderated,n_updates,updates,_21,text,condition,ICD10,name,gender,age,ethnicity,urgency,sentiment,truth,notes,smiling,guess_ethnicity,guess-ethnicity,guess,Sentiment,guess ethnicity,Target_completion
488,495,Help me support my sick father,Raising money to see my sick father,https://static.givealittle.co.nz/assets/hero/cfa715d1-e709-4367-a625-b20f01636ccb-320:False,https://givealittle.co.nz/cause/help-me-support-my-sick-father,1555.0,2000.0,closed,Read more,https://givealittle.co.nz/cause/help-me-support-my-sick-father,20 Oct 2024,27 Oct 2024,0,"Greenhithe, Auckland",39.0,Hey guys. I'm raising money in order to travel down to Nelson to see my father who has been diag...,Travel funds to get from Auckland to Nelson and back.,"Created by, and paying to a verified bank account of,_x000D_\nJoshua Westley",Page Moderated,NaN,NaN,NaN,Help me support my sick father Raising money to see my sick father Hey guys. I'm raising money i...,Alzheimer's,F03,Dad,Male,Not specified,European,90.0,100.0,80.0,"The text does not mention the person's age or ethnicity. Based on the image, the person appears ...",True,NaN,NaN,NaN,NaN,NaN,77.750000
489,496,My Cambodian Sister Fighting the Big C,Please help my Cambodian sister fight the big C.,https://static.givealittle.co.nz/assets/hero/e17703d2-5687-45ab-9276-b20b0085e2e1-320:False,https://givealittle.co.nz/cause/my-cambodian-sister-fighting-the-big-c,2000.0,3000.0,closed,Read more,https://givealittle.co.nz/cause/my-cambodian-sister-fighting-the-big-c,16 Oct 2024,29 Oct 2024,0,Nationwide,26.0,"Last year, Channa was my tour guide in Cambodia. Having grown up in similar political systems an...","rent, groceries, transport, and other daily necessities","Created by, and paying to a verified bank account of,_x000D_\n _x000D_\n Antje Decke...",Page Moderated,1.0,Thank you!!!\n1 November 2024\nThank you for opening your heart and wallet for Channa. I spoke t...,NaN,My Cambodian Sister Fighting the Big C Please help my Cambodian sister fight the big C. Last yea...,Breast Cancer,C50,Channa,Female,Adult,Asian,100.0,100.0,80.0,The text is a personal and emotional appeal from the campaign creator to help a friend in need. ...,True,NaN,NaN,NaN,NaN,NaN,66.666667
491,498,Flights needed NZ-London to help 83 year old mum through a mastectomy,Raising funds to send a caring daughter to London to look after her 83 year old mum following a ...,https://static.givealittle.co.nz/assets/hero/03a34b2c-1401-4460-8ca3-b20800b6f41e-320:False,https://givealittle.co.nz/cause/flights-needed-nz-london-to-help-83-year-old-mum,0.0,3340.0,closed,Read more,https://givealittle.co.nz/cause/flights-needed-nz-london-to-help-83-year-old-mum,13 Oct 2024,16 Oct 2024,0,NaN,NaN,My mother has breast cancer and at 83 is having a mastectomy on 24th October. She lives alone an...,Just the flights,"Created by, and paying to a verified bank account of,_x000D_\n _x000D_\n Sian Jones",Page Moderated,NaN,NaN,NaN,Flights needed NZ-London to help 83 year old mum through a mastectomy Raising funds to send a ca...,breast cancer,C50,unknown,unknown,83,white,90.0,80.0,80.0,"The ethnicity is not mentioned in the text, but based on the image, I guess the person is white....",True,NaN,NaN,NaN,NaN,NaN,0.000000
493,500,Tom’s recovery from traumatic brain injury.,Raising funds for Tom’s arduous recovery journey,https://static.givealittle.co.nz/assets/hero/b96d62a6-aaf0-47f1-9db5-b20700b7b7ac-320:False,https://givealittle.co.nz/cause/toms-recovery-from-traumatic-brain-injury,2680.0,5000.0,closed,Read more,https://givealittle.co.nz/cause/toms-recovery-from-traumatic-brain-injury,12 Oct 2024,24 Oct 2024,0,Nelson / Tasman,22.0,On Monday 7th October 2024 Tom was on his way to work on his quad bike when he came across a her...,"We have started this page to help our parents with accomodation, travel and food costs while the...",

In [3]:
df.smiling = df.smiling.str.lower().replace({
    "true": True,
    "false": False,
    "yes": True,
    "no": False,
    "not applicable": None,
    "unknown": None,
    "undefined": None
})

In [4]:
df.gender = df.gender.str.lower().where(df.gender.str.lower().isin(['male', 'female']), 'other')
df.gender.value_counts()

gender
female    2074
male      1546
other      447
Name: count, dtype: int64

In [5]:
df.ethnicity = df.ethnicity.str.lower().replace({
    "caucasian": "white",
    "likely caucasian": "white",
    "likely white": "white",
    "likely of european descent": "white",
    "likely caucasian based on the image": "white",
    "likely european": "white",
    "european": "white",
    "maori": "māori",
    "not mentioned": None,
    "not specified": None,
    "unspecified": None,
    "not found": None,
    "not provided": None,
    "unknown": None,
})
top_10 = df.ethnicity.value_counts().head(10)
df.ethnicity = df.ethnicity.where(df.ethnicity.isin(top_10.index), "other")

In [10]:
df.condition.str.lower().value_counts().head(20)

condition
cancer                225
breast cancer         173
gender dysphoria      109
stroke                 65
bowel cancer           62
multiple sclerosis     59
lung cancer            55
melanoma               40
mental health          37
obesity                37
endometriosis          34
cerebral palsy         34
kidney failure         31
type 1 diabetes        30
heart attack           28
infertility            28
ovarian cancer         25
unknown                25
terminal cancer        23
cardiac arrest         22
Name: count, dtype: int64

In [26]:
smf.ols(formula="Target_completion ~ sentiment + smiling + gender + truth", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Target_completion   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     2.431
Date:                Mon, 16 Dec 2024   Prob (F-statistic):             0.0344
Time:                        10:16:21   Log-Likelihood:                -2277.8
No. Observations:                 421   AIC:                             4568.
Df Residuals:                     415   BIC:                             4592.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          25.2501     17.544      1.439      0.151      -9.236      59.737
smiling[T.True]    11.7268      5.606      2.092      0.037       0.707      22.746
gender[T.male]      9.4733      5.590      1.695      0.091      -1.514      20.461
gender[T.other]     2.2159     11.663      0.190      0.849     -20.711      25.142
sentiment          -0.3565      0.139     -2.565      0.011      -0.630      -0.083
truth               0.3923      0.225      1.743      0.082      -0.050       0.835
==============================================================================
Omnibus:                      447.003   Durbin-Watson:                   2.107
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28466.226
Skew:                           4.583   Prob(JB):                         0.00
Kurtosis:                      42.227   Cond. No.                         756.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
smf.ols(formula="Target_completion ~ sentiment + smiling + gender + truth + urgency + ethnicity", data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Target_completion   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.717
Date:                Mon, 16 Dec 2024   Prob (F-statistic):             0.0498
Time:                        10:02:42   Log-Likelihood:                -2271.8
No. Observations:                 421   AIC:                             4574.
Df Residuals:                     406   BIC:                             4634.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        26.4122     17.882      1.477      0.140      -8.741      61.565
smiling[T.True]                   6.0377      5.931      1.018      0.309      -5.622      17.697
gender[T.male]                   11.5677      5.695      2.031      0.043       0.373      22.762
gender[T.other]                  -0.8649     12.253     -0.071      0.944     -24.952      23.222
ethnicity[T.asian]                6.1609     13.648      0.451      0.652     -20.668      32.990
ethnicity[T.black]              -11.2939     35.120     -0.322      0.748     -80.333      57.746
ethnicity[T.fijian]             -21.1519     35.067     -0.603      0.547     -90.087      47.783
ethnicity[T.filipino]            -7.9847     17.862     -0.447      0.655     -43.098      27.129
ethnicity[T.indian]              -7.8303     12.386     -0.632      0.528     -32.178      16.518
ethnicity[T.māori]               19.4902     15.153      1.286      0.199     -10.298      49.279
ethnicity[T.other]                8.7906      8.971      0.980      0.328      -8.845      26.426
ethnicity[T.pacific islander]    24.9362     12.777      1.952      0.052      -0.181      50.053
ethnicity[T.south asian]       -1.21e-15   2.09e-15     -0.579      0.563   -5.32e-15     2.9e-15
ethnicity[T.white]               15.2950      7.480      2.045      0.042       0.590      30.000
sentiment                        -0.2408      0.149     -1.613      0.108      -0.534       0.053
truth                             0.3866      0.231      1.675      0.095      -0.067       0.840
urgency                          -0.2515      0.146     -1.721      0.086      -0.539       0.036
==============================================================================
Omnibus:                      455.582   Durbin-Watson:                   2.077
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            31394.200
Skew:                           4.702   Prob(JB):                         0.00
Kurtosis:                      44.246   Cond. No.                     3.83e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.31e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
model = RandomForestRegressor()
cols = "sentiment + smiling + gender + truth + urgency + ethnicity".split(" + ")
X = pd.get_dummies(df[cols])
y = df["Target_completion"]
model.fit(X, y)

RandomForestRegressor()

In [20]:
list(zip(cols, model.feature_importances_))

[('sentiment', 0.29360972579137923),
 ('smiling', 0.1918739363914363),
 ('gender', 0.21766417627244206),
 ('truth', 0.03354045864811954),
 ('urgency', 0.00725001053236176),
 ('ethnicity', 0.0019828296882488)]